In [1]:
import pandas as pd
import numpy as np
import codecs
from tqdm import tqdm
import re
from datetime import datetime

In [7]:
df = pd.read_csv("data/南海自由航行_news_newlabel.csv")

In [25]:
event_info = {}
eventtoid = {}
idtoevent = {}
id_num = 0
with codecs.open("dict/事件节点.txt","r","UTF-8") as rf:
    for line in rf.readlines():
        event_center, event_des = line.strip().split(":")
        event_info[event_center] = event_des
        eventtoid[event_center] = id_num
        idtoevent[id_num] = event_center
        id_num += 1
eventtoid

{'入侵行动': 0, '防卫行动': 1, '访问行动': 2, '军演行动': 3, '国家立场': 4, '媒体评论': 5}

In [26]:
# 利用二维列表表示关系图
event_graph = np.zeros([id_num, id_num])
event_graph

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# 根据新闻标题的内容共现关系构建事件中心关系图

- 根据标题的内容标签共现构建关系图

In [27]:
# 依据新闻标题构建事件中心关系图
for content_label_info in tqdm(df['content_label']):
    labels = content_label_info.split(' ')
    # print(labels)
    for i in range(len(labels)-1):
        for j in range(i+1, len(labels)):
            event_graph[eventtoid[labels[i]]][eventtoid[labels[j]]] += 1
            event_graph[eventtoid[labels[j]]][eventtoid[labels[i]]] += 1
event_graph

100%|██████████████████████████████████████████████████████████████████████████| 4943/4943 [00:00<00:00, 411913.79it/s]


array([[  0., 152.,   1.,  42., 290., 148.],
       [152.,   0.,   1.,   8.,  99.,  46.],
       [  1.,   1.,   0.,   0.,   3.,   6.],
       [ 42.,   8.,   0.,   0.,  28.,  34.],
       [290.,  99.,   3.,  28.,   0.,  81.],
       [148.,  46.,   6.,  34.,  81.,   0.]])

# 根据新闻内容构建事件中心关系图

- 根据文章共现关系构建关系图,分句-分类-构建
- 根据文章前后句连接词构建关系图 分句-分类-识别连接词-构建

In [14]:
# 依据新闻内容构建事件中心关系图
iter_num = 0
for content in df['content']:
    # content = ''.join(content.split()) #split方法中不带参数时，表示分割所有换行符、制表符、空格
    # content = content.replace(u'\u3000', u'').replace('\n','').replace(' ','').replace('\r','')
    # content_words = re.split('。|！|？',content)
    content_words = content.split()
    # print(content_words)
    if iter_num > 10: break
    iter_num += 1

['原标题：美方在南海开展“航行自由行动”？中方：人不犯我', '我不犯人']
['外交部：坚决反对打着航行自由幌子损害南海沿岸国主权与安全利益的行为', '外交部发言人华春莹16日在例行记者会上表示，中国坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '在当日的记者会上，有记者问：美国副总统彭斯说南海不属于任何一个国家，美国将继续在国际法允许的范围内行使航行和飞越自由。中方是否认为彭斯的相关言论是种挑衅？', '华春莹说，南海航行和飞越自由从来不存在任何问题。“我们曾经多次问美方或者声称南海航行自由有问题的人，请拿出证据来，证明有哪个国家的哪艘船、哪架飞机、什么时候在南海航行或者飞越时遇到了问题？但到目前为止，没有任何人给出一个明确的答案。由此可见，所谓南海航行与飞越自由根本就是一个不存在的伪命题。”', '华春莹表示，中方的立场非常明确，愿与地区国家一道，坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对那些打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '“另外，请你提醒一下美方有关人士，美国到现在还没有批准《联合国海洋法公约》。如果美方能够尽早批准并遵守《公约》，我想将更加有利于维护南海地区的和平稳定。”华春莹说。', '消息来源：新华社', '本期编审：孙', '利', '责任编辑：朱西迪、李明', '投稿邮箱：gfsk@cnr.cn', '转载请注明出处！']
['一艘英国皇家海军军舰日前驶向越南途中，在接近中国于南海声索的岛屿附近航行，以彰显“航行自由”，但遭中国方面派出舰船及直升机驱离。']
['外交部发言人华春莹16日在例行记者会上表示，中国坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '在当日的记者会上，有记者问：美国副总统彭斯说南海不属于任何一个国家，美国将继续在国际法允许的范围内行使航行和飞越自由。中方是否认为彭斯的相关言论是种挑衅？', '华春莹说，南海航行和飞越自由从来不存在任何问题。“我们曾经多次问美方或者声称南海航行自由有问题的人，请拿出证据来，证明有哪个国家的哪艘船、哪架飞机、什么时候在南海航行或者飞越时遇到了问题？但到目前为止，没有任何人给出

# 效果可视化

In [34]:
from pyecharts import options as opts
from pyecharts.charts import Graph


nodes_data = [opts.GraphNode(name=event_info[event_key], symbol_size=40) for event_key in eventtoid.keys()]
links_data = []

[rows, cols] = event_graph.shape
for i in range(rows):
    for j in range(cols):
        if i == j: continue
        if event_graph[i, j] == 0: continue
        links_data.append(opts.GraphLink(source=event_info[idtoevent[i]], target=event_info[idtoevent[j]], value=event_graph[i, j]))

graph = (
    Graph()
    .add(
        "",
        nodes_data,
        links_data,
        repulsion=4000,
        edge_label=opts.LabelOpts(
            is_show=True, position="middle", formatter="{c}"
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="南海事件")
    )
    # .render("result/event_center_graph.html")
)
graph.render_notebook()

# 事件-时间演化图构建

In [3]:
df = pd.read_csv("data/南海自由航行_news_newlabel.csv")

In [4]:
df['time'] = pd.to_datetime(df['time'])
df['time'] = [datetime.strftime(x,'%Y-%m-%d') for x in df['time']]
# df = df.set_index('time')
df = df[(df['time'] >= '2019-11-20') & (df['time'] <= '2019-11-27')] # 筛选时间段
df['time']

69      2019-11-22
194     2019-11-23
344     2019-11-20
384     2019-11-23
450     2019-11-27
           ...    
4796    2019-11-25
4850    2019-11-27
4851    2019-11-27
4853    2019-11-27
4854    2019-11-27
Name: time, Length: 119, dtype: object

In [5]:
group_result = df.groupby (df['time'])

In [12]:
from cluster_vps import k_means_tfidf
import codecs
import json
json_result = {}  # 输出为json文件反馈给前端
with codecs.open("result/news_date_k_means_clusters.txt", "w", "utf-8") as wf:
    for time, day_df in group_result:
        print(time)
        json_result[time] = []
        
        title_list = day_df['title'].tolist()
        result = k_means_tfidf(title_list, 1, 5)
        wf.write(time + "\n")
        for key in result[0].keys():
            tmp = {}
            wf.write("cluster" + str(key) + "\n")
            wf.write("center: " + result[1][key] + "\n")
            tmp['cluster'] = str(key)
            tmp['center'] = result[1][key]
            tmp['title_list'] = []
            for i in result[0][key]:
                wf.write(title_list[i] + "\n")
                tmp['title_list'].append(title_list[i])
            wf.write("\n")
            json_result[time].append(tmp)
        wf.write("=============\n\n")
with codecs.open("result/20191120-20191127_titlecluster.json", "w", 'utf-8') as wf:
    json.dump(json_result, wf, indent=4)

2019-11-20
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.03494684619278409
2019-11-22
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.24097830897037528
2019-11-23
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.30804076463107805
2019-11-24
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.3288123460202369
2019-11-25
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.003755232233632305
2019-11-26
2019-11-27
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.35822082352291645


In [6]:
import pickle
pkl_rf = open('data/南海_20191120_20191127.pkl','rb')
result = pickle.load(pkl_rf)
# result['view_cluster_data'][0]['center']='兔子急了，也是会咬人的。'
print(result['view_cluster_data'][0])
# pkwf = open('data/南海_20191120_20191127.pkl',"wb") 
# pickle.dump(result, pkwf) 

{'cluster': '2', 'center': '兔子急了，也是会咬人的。', 'view_num': 153, 'view_list': ['美方欢迎中国留学生赴美国留学。', '美方一贯的立场，并无其他新的说法。', '“联合防空作战训练”的目的主要在结合军、民防空系统及联战指挥机制运作，检视台军重要目标防护能力，并整合三军防空武力及资电战力，精进联合防空作战效能。', '隐身的时代，F/A-18“大黄蜂”还能凭借挂载大型副油箱将澳大利亚空军的打击火力投送到东帝汶——因为澳大利亚因为帝汶海底石油资源存在归属争议。', '类似的提法。', '中国很可能会派遣一个石油钻井平台，到“海洋地质8号”在越南专属经济区进行勘探的地区，进行钻探。', '其捍卫这条国际贸易通道自由的决心。', '战机升空是为联翔操演所需，而这项操演是定期举行，并非针对中国大陆航母穿越台海而做的安排。', '中国领导人表示，我们愿在平等和相互尊重的基础上，同美方达成第一阶段经贸协议。', '美国所谓“自由航行”行动越来越成为独角戏，南海周边国家不愿为美国的一己之私背书，更不想被美国绑上联合制华的战车。', '美国退役陆军中将艾肯伯里表示，两国在南海好几次几乎出现军事对峙，随着美中关系继续恶化，两军出现军事意外或者作战误判的风险日益紧迫。', '杜特尔特低调处理南海问题、坚决改善与中国的关系的政策是正确的。', '“中方敦促美方停止在南海炫耀武力，不要挑起和升级南海的紧张局势。”', '不愿在中美之间选边站，希望美国能够尊重中国和东盟国家共同维护南海和平稳定的努力。', '了，兔子急了，也是会咬人的。', '魏凤和就台湾、南海、涉港等问题重申了中方立场。', '尾盘存挟淡仓情况；香港本地事件虽未见好转，但在国家领导人发话之后预计未来出现进一步恶化概率在降低。', '美国企图遏制中俄是大错特错。', '了新的国家国防战略，表达了对来自中国的安全挑战的重视，美国与菲律宾的条约应该反映这项变化。', '越南一名高级官员11月6日表示，越南将寻求采取法律行动解决与中国在南海的主权纠纷。', '“我们试图发出的明确信号是，我们并不反对中国本身，而是我们都支持国际法，我们认为中国也应该遵守国际法。”', '在自家门口训练很正常。', '这是俺们美国人的资产，没有俺们的允许，外人不得入内，加拿大人也不可以

# 事件预测字典构建

In [2]:
event_pre_dict = {}
event_pre_dict['南海'] = {'美国在南海挑起争端':[{'新冠':5}]}
event_pre_dict['朝核'] = {'朝鲜试射导弹':[{'制裁':5}]}
event_pre_dict['台选'] = {'台湾政局鼓吹台独言论':[{'蔡英文':5}]}

In [3]:
import json
with codecs.open('dict/EventPre.json','w','utf-8') as wf:
    json.dump(event_pre_dict, wf, indent=4, ensure_ascii=False)

# 事件趋势预测

In [4]:
news_df = pd.read_csv("data/南海_202007_news_newdata.csv")[['time','crisis']]

In [5]:
news_df['time'] = pd.to_datetime(news_df['time']) # 将数据类型转换为日期类型
news_df = news_df.set_index('time') # 将time设置为index

In [6]:
news_df.resample('M').count().head()

,crisis
time,
2018-01-31 00:00:00+00:00,41
2018-02-28 00:00:00+00:00,25
2018-03-31 00:00:00+00:00,160
2018-04-30 00:00:00+00:00,681
2018-05-31 00:00:00+00:00,440


In [7]:
date_list = []
crisis_list = []

for index, row in news_df.resample('D').sum().iterrows():
    date_list.append(index.strftime('%Y-%m-%d'))
    crisis_list.append(row['crisis'])

In [16]:
import pyecharts.options as opts
from pyecharts.charts import Line
print(crisis_list)
graph = (
    Line()
    .add_xaxis(xaxis_data=date_list[-100:-1])
    .add_yaxis(
        series_name="crisis value",
        y_axis=crisis_list[-100:-1],
        symbol="emptyCircle",
        is_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=True),
    )

)
graph.render_notebook()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 52, 100, 0, 0, 0, 94, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 57, 53, 0, 0, 0, 0, 99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 89, 0, 0, 60, 0, 0, 0, 0, 0, 0, 0, 0, 57, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 778, 540, 0, 636, 0, 180, 519, 176, 0, 0, 162, 103, 430, 0, 0, 0, 42, 0, 807, 314, 449, 0, 0, 0, 0, 50, 181, 24, 75, 0, 31, 54, 0, 0, 86, 0, 0, 385, 0, 0, 0, 58, 142, 0, 66, 0, 52, 30, 32, 112, 155, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 123, 0, 59, 0, 730, 642, 891, 529, 739, 0, 221, 0, 87, 195, 343, 262, 56, 0, 20, 167, 467, 0, 81, 0, 0, 0, 0, 260, 99, 0, 0, 134, 31, 49, 0, 83, 243, 154, 0, 0, 384, 156, 186, 590, 530, 0, 0, 139, 74, 0, 0, 93, 0, 0, 0, 0, 0, 56, 0, 0, 0, 0, 0, 0, 0, 129, 0, 0, 0, 0, 222, 28, 0, 76, 0, 60, 184, 95, 32, 111, 149, 146, 91, 93, 0, 0, 0, 0, 0, 31, 0, 0, 331, 87, 0, 0, 0, 0, 0, 564, 0, 449, 21, 474, 40, 36, 1476, 1710, 278, 580, 568, 417, 94, 55, 0, 0, 0, 363, 414, 130, 0, 117, 127, 0, 0, 0, 83, 21, 457, 117, 189, 695, 3865, 

In [9]:
count_date_list = []
count_list = []
for index, row in news_df.resample('D').count().iterrows():
    count_date_list.append(index.strftime('%Y-%m-%d'))
    count_list.append(float(row['crisis']))

In [10]:
import pyecharts.options as opts
from pyecharts.charts import Line

count_graph = (
    Line()
    .add_xaxis(xaxis_data=count_date_list)
    .add_yaxis(
        series_name="count value",
        y_axis=count_list,
        symbol="emptyCircle",
        is_symbol_show=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
)
count_graph.render_notebook()

In [12]:
pre_value_list = []
avg_crisis_list = []
for i in range(0, len(date_list)):
    if count_list[i] == 0:
        avg_crisis_list.append(0)
    else:
        avg_crisis_list.append(float(crisis_list[i]/count_list[i]))
    
    if i == 0:
        pre_value_list.append(count_list[i])
    elif count_list[i-1] == 0:
        # print(count_list[i-1])
        # print(crisis_list[i-1])
        pre_value_list.append(count_list[i-1])
    else:
        pre_value_list.append(count_list[i-1] + float(crisis_list[i-1]/count_list[i-1]))

In [13]:
print(avg_crisis_list)
# print(count_list)
# print(pre_value_list)

[0.0, 0, 0, 0, 0.0, 0, 0.0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 8.0, 7.428571428571429, 33.333333333333336, 0.0, 0.0, 0.0, 47.0, 0.0, 0.0, 0.0, 0, 0, 0.0, 0.0, 0, 0.0, 0, 0, 57.0, 17.666666666666668, 0, 0, 0, 0, 33.0, 0.0, 0.0, 0, 0, 0.0, 0.0, 0, 0, 0.0, 0.0, 89.0, 0.0, 0.0, 20.0, 0, 0.0, 0.0, 0, 0, 0, 0, 0.0, 57.0, 45.0, 0, 0, 0.0, 0, 0, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 70.72727272727273, 60.0, 0, 24.46153846153846, 0.0, 6.923076923076923, 11.042553191489361, 11.0, 0, 0, 12.461538461538462, 7.357142857142857, 12.647058823529411, 0.0, 0.0, 0.0, 2.210526315789474, 0.0, 20.175, 16.526315789473685, 4.276190476190476, 0.0, 0, 0, 0.0, 3.5714285714285716, 6.961538461538462, 1.3333333333333333, 2.8846153846153846, 0.0, 5.166666666666667, 0.72, 0.0, 0.0, 1.5636363636363637, 0.0, 0.0, 16.73913043478261, 0.0, 0.0, 0.0, 1.6571428571428573, 8.875, 0.0, 8.25, 0.0, 7.428571428571429, 4.285714285714286, 6.4, 12.444444444444445, 19.375, 3.875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.666666666666666

In [14]:
import pyecharts.options as opts
from pyecharts.charts import Line

count_graph = (
    Line()
    .add_xaxis(xaxis_data=count_date_list[-100:-1])
    .add_yaxis(
        series_name="count value",
        y_axis=count_list[-100:-1],
        symbol="emptyCircle",
        is_symbol_show=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="pre value",
        y_axis=pre_value_list[-100:-1],
        symbol="emptyCircle",
        is_symbol_show=True,
        label_opts=opts.LabelOpts(is_show=False),
    )
)
count_graph.render_notebook()